# 데이터 집계와 그룹 연산

배우게 될 내용
1. 하나 이상의 키를 이용해서 pandas 객체를 여러 조각으로 나누는 방법
2. 합계, 평균, 표준편차, 사용자 정의 함수 같은 그룹 요약 통계를 계산하는 방법
3. 정규화, 선형회귀, 등급 또는 부분집합 선택 같은 집단 내 변형이나 다른 조작을 적용하는 방법
4. 피벗테이블과 교차일람표를 구하는 방법
5. 변위치 분석과 다른 통계 집단 분석을 수행하는 방법

1. Groupby 메카닉 
- 분리-적용-결합 split-apply-combine

In [22]:
import pandas as pd

In [23]:
import numpy as np

In [24]:
df = pd.DataFrame({'key1':['a','a','b','b','a'],
                   'key2':['one','two','one','two','one'],
                   'data1':np.random.randn(5),
                   'data2':np.random.randn(5)})                   

In [25]:
df

,key1,key2,data1,data2
0,a,one,0.056403,0.568546
1,a,two,-0.292824,1.092401
2,b,one,-1.654920,0.349670
3,b,two,-1.884347,-0.043699
4,a,one,0.470910,0.089319


In [26]:
grouped = df['data1'].groupby(df['key1'])

In [27]:
grouped

In [28]:
grouped.mean()

key1
a    0.078163
b   -1.769633
Name: data1, dtype: float64

In [29]:
means = df['data1'].groupby([df['key1'],df['key2']]).mean()

In [30]:
means

key1  key2
a     one     0.263657
      two    -0.292824
b     one    -1.654920
      two    -1.884347
Name: data1, dtype: float64

In [31]:
means.unstack()

key2,one,two
key1,,
a,0.263657,-0.292824
b,-1.654920,-1.884347


In [32]:
states=np.array(['Ohio','California','California','Ohio','Ohio'])
years=np.array([2005,2005,2006,2005,2006])
df['data1'].groupby([states,years]).mean()

California  2005   -0.292824
            2006   -1.654920
Ohio        2005   -0.913972
            2006    0.470910
Name: data1, dtype: float64

In [33]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.078163,0.583422
b,-1.769633,0.152985


In [34]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   0.263657  0.328932
     two  -0.292824  1.092401
b    one  -1.654920  0.349670
     two  -1.884347 -0.043699

In [35]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 그룹 간 순회하기

In [36]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.056403  0.568546
1    a  two -0.292824  1.092401
4    a  one  0.470910  0.089319
b
  key1 key2     data1     data2
2    b  one -1.654920  0.349670
3    b  two -1.884347 -0.043699


In [37]:
for (k1,k2), group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.056403  0.568546
4    a  one  0.470910  0.089319
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.292824  1.092401
('b', 'one')
  key1 key2    data1    data2
2    b  one -1.65492  0.34967
('b', 'two')
  key1 key2     data1     data2
3    b  two -1.884347 -0.043699


In [38]:
pieces=dict(list(df.groupby('key1')))

In [39]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-1.654920,0.349670
3,b,two,-1.884347,-0.043699


In [40]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [41]:
for dtype, group in grouped:
    print(dtype)
    print(group)

a
0    0.056403
1   -0.292824
4    0.470910
Name: data1, dtype: float64
b
2   -1.654920
3   -1.884347
Name: data1, dtype: float64


In [42]:
df.groupby('key1')['data1']

In [43]:
df['data1'].groupby(df['key1'])

In [44]:
df[['data2']].groupby(df['key1'])

In [45]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.328932
     two   1.092401
b    one   0.349670
     two  -0.043699

In [46]:
s_grouped = df.groupby(['key1','key2'])['data2']

In [47]:
s_grouped

In [48]:
s_grouped.mean()

key1  key2
a     one     0.328932
      two     1.092401
b     one     0.349670
      two    -0.043699
Name: data2, dtype: float64

In [50]:
people=pd.DataFrame(np.random.randn(5,5),
                    columns=['a','b','c','d','e'],
                    index=['Joe','Steve','Wes','Jim','Travis'])

In [51]:
people.iloc[2:3,[1,2]]=np.nan #nan값을 추가하자.

In [52]:
people

,a,b,c,d,e
Joe,-0.367850,1.875924,-0.153505,0.094824,-0.012996
Steve,-0.069113,0.600483,-0.190527,-0.020094,-1.214501
Wes,-0.316767,NaN,NaN,-1.898933,0.128946
Jim,0.179120,0.287912,-0.748387,1.499346,-0.481831
Travis,0.411864,-2.012535,0.260852,-1.623552,1.106466


In [56]:
people.iloc[0:1,0]

Joe   -0.36785
Name: a, dtype: float64

In [57]:
mapping={'a':'red','b':'red','c':'blue',
         'd':'blue','e':'red','f':'orange'}

In [58]:
by_column=people.groupby(mapping,axis=1)

In [59]:
by_column.sum()

,blue,red
Joe,-0.058680,1.495078
Steve,-0.210621,-0.683131
Wes,-1.898933,-0.187821
Jim,0.750959,-0.014798
Travis,-1.362700,-0.494206


In [63]:
map_series=pd.Series(mapping)

In [64]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [65]:
people.groupby(map_series,axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [66]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.505497,2.163836,-0.901892,-0.304762,-0.365881
5,-0.069113,0.600483,-0.190527,-0.020094,-1.214501
6,0.411864,-2.012535,0.260852,-1.623552,1.106466


In [67]:
key_list=['one','one','one','two','two']

In [68]:
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one -0.367850  1.875924 -0.153505 -1.898933 -0.012996
  two  0.179120  0.287912 -0.748387  1.499346 -0.481831
5 one -0.069113  0.600483 -0.190527 -0.020094 -1.214501
6 two  0.411864 -2.012535  0.260852 -1.623552  1.106466

In [71]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],[1,3,5,1,3]],names=['cty','tenor'])

In [72]:
hier_df = pd.DataFrame(np.random.randn(4,5),columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.067195  0.140678 -2.141335 -0.666872 -1.725308
1     -2.369873 -1.644435  0.805632 -0.635853  0.325159
2     -1.353974 -1.499354 -0.175193 -1.135723 -2.104290
3     -1.057988  0.855798 -1.089192 -1.662830 -1.018137

In [73]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [75]:
df

,key1,key2,data1,data2
0,a,one,0.056403,0.568546
1,a,two,-0.292824,1.092401
2,b,one,-1.654920,0.349670
3,b,two,-1.884347,-0.043699
4,a,one,0.470910,0.089319


In [76]:
grouped = df.groupby('key1')

In [78]:
grouped['data1'].quantile(0.9)

key1
a    0.388009
b   -1.677863
Name: data1, dtype: float64

In [79]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [80]:
grouped.agg(peak_to_peak)

C:\Users\gyurihan\AppData\Local\Temp\ipykernel_13012\4193407341.py:1: FutureWarning: ['key2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped.agg(peak_to_peak)


,data1,data2
key1,,
a,0.763735,1.003082
b,0.229426,0.393369


In [81]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.078163  0.382332 -0.292824 -0.118211  0.056403  0.263657   
b      2.0 -1.769633  0.162229 -1.884347 -1.826990 -1.769633 -1.712277   

              data2                                                    \
          max count      mean       std       min       25%       50%   
key1                                                                    
a     0.47091   3.0  0.583422  0.501706  0.089319  0.328932  0.568546   
b    -1.65492   2.0  0.152985  0.278154 -0.043699  0.054643  0.152985   

                          
           75%       max  
key1                      
a     0.830473  1.092401  
b     0.251327  0.349670

In [84]:
tips=pd.read_csv('../data/examples/tips.csv')

In [88]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [92]:
tips.count

<bound method DataFrame.count of      total_bill   tip smoker   day    time  size   tip_pct
0         16.99  1.01     No   Sun  Dinner     2  0.059447
1         10.34  1.66     No   Sun  Dinner     3  0.160542
2         21.01  3.50     No   Sun  Dinner     3  0.166587
3         23.68  3.31     No   Sun  Dinner     2  0.139780
4         24.59  3.61     No   Sun  Dinner     4  0.146808
..          ...   ...    ...   ...     ...   ...       ...
239       29.03  5.92     No   Sat  Dinner     3  0.203927
240       27.18  2.00    Yes   Sat  Dinner     2  0.073584
241       22.67  2.00    Yes   Sat  Dinner     2  0.088222
242       17.82  1.75     No   Sat  Dinner     2  0.098204
243       18.78  3.00     No  Thur  Dinner     2  0.159744

[244 rows x 7 columns]>

In [86]:
# total_bill 에서 팁의 비율을 추가하자.
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [87]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [93]:
grouped = tips.groupby(['day', 'smoker'])

In [94]:
grouped

In [95]:
grouped_pct = grouped['tip_pct']

In [96]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [97]:
grouped_pct.agg(['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [98]:
grouped_pct.agg([('foo','mean'),('bar',np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [99]:
functions = ['count','mean','max']

In [100]:
result = grouped['tip_pct','total_bill'].agg(functions)

C:\Users\gyurihan\AppData\Local\Temp\ipykernel_13012\2281889977.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct','total_bill'].agg(functions)


In [101]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [102]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [103]:
ftuples = [('Durchschnitt','mean'),('Abweichung',np.var)]

In [104]:
grouped['tip_pct','total_bill'].agg(ftuples)

C:\Users\gyurihan\AppData\Local\Temp\ipykernel_13012\1934420057.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct','total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [105]:
grouped.agg({'tip':np.max,'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [106]:
grouped.agg({'tip_pct':['min','max','mean','std'],
             'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [107]:
tips.groupby(['day','smoker'],as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [113]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [114]:
top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [115]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [118]:
tips.groupby(['smoker','day']).apply(top,n=1,column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [119]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [120]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [121]:
result.unstack()

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [125]:
f = lambda x : x.describe()
grouped.apply(f)


#grouped.apply(lambda x : x.describe())

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

In [124]:
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [127]:
frame = pd.DataFrame({'data1':np.random.randn(1000),
                      'data2':np.random.randn(1000)})

In [128]:
quartiles = pd.cut(frame.data1, 4)

In [129]:
quartiles[:10]

0     (-0.106, 1.554]
1     (-0.106, 1.554]
2      (1.554, 3.215]
3     (-0.106, 1.554]
4      (1.554, 3.215]
5     (-0.106, 1.554]
6     (-0.106, 1.554]
7    (-1.766, -0.106]
8    (-1.766, -0.106]
9     (-0.106, 1.554]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.433, -1.766] < (-1.766, -0.106] < (-0.106, 1.554] < (1.554, 3.215]]

In [130]:
def get_stats(group):
    return {'min':group.min(),'max':group.max(),
            'count':group.count(),'mean':group.mean()}

In [131]:
grouped = frame.data2.groupby(quartiles)

In [132]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.433, -1.766]",-2.248248,1.715527,37.0,-0.374837
"(-1.766, -0.106]",-3.174194,3.063836,420.0,-0.004679
"(-0.106, 1.554]",-2.994952,3.435548,488.0,-0.013110
"(1.554, 3.215]",-2.073960,2.343378,55.0,-0.026395


In [134]:
#변위치를 숫자로 변환
grouping = pd.cut(frame.data1, 10, labels=False)

In [135]:
grouped = frame.data2.groupby(grouping)

In [136]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-0.898527,-0.753768,3.0,-0.828209
1,-1.509416,1.715527,7.0,-0.424315
2,-2.458766,2.254035,60.0,-0.319566
3,-3.174194,2.348432,145.0,0.011831
4,-2.132420,3.063836,242.0,0.029252
5,-2.994952,3.435548,240.0,0.016946
6,-2.442568,2.415851,203.0,-0.080030
7,-1.874995,1.891518,64.0,0.062472
8,-1.422362,2.343378,29.0,0.111802


In [137]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan

In [138]:
s

0         NaN
1   -0.331778
2         NaN
3    0.609103
4         NaN
5    0.180672
dtype: float64

In [139]:
s.fillna(s.mean())

0    0.152666
1   -0.331778
2    0.152666
3    0.609103
4    0.152666
5    0.180672
dtype: float64

In [140]:
states = ['Ohio','New York','Vermont','Florida',
          'Oregon','Nevada','California','Idaho']

In [159]:
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [141]:
group_key = ['East']*4 + ['West']*4

In [142]:
data = pd.Series(np.random.randn(8),index=states)
data

Ohio          0.911372
New York     -0.346453
Vermont       0.180745
Florida      -2.205025
Oregon        1.103732
Nevada       -0.026118
California    0.549910
Idaho        -1.387963
dtype: float64

In [143]:
data[['Vermont','Nevada','Idaho']]=np.nan

In [144]:
data

Ohio          0.911372
New York     -0.346453
Vermont            NaN
Florida      -2.205025
Oregon        1.103732
Nevada             NaN
California    0.549910
Idaho              NaN
dtype: float64

In [145]:
fill_mean = lambda g: g.fillna(g.mean())

In [146]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.911372
New York     -0.346453
Vermont      -0.546702
Florida      -2.205025
Oregon        1.103732
Nevada        0.826821
California    0.549910
Idaho         0.826821
dtype: float64

In [148]:
fill_values={'East':0.5,'West':-1}
fill_func = lambda g : g.fillna(fill_values[g.name])

In [149]:
data.groupby(group_key).apply(fill_func)

Ohio          0.911372
New York     -0.346453
Vermont       0.500000
Florida      -2.205025
Oregon        1.103732
Nevada       -1.000000
California    0.549910
Idaho        -1.000000
dtype: float64

In [152]:
#하트, 스페이드, 클럽, 다이아몬드
suits = ['H','S','C','D']
card_val = (list(range(1,11))+[10]*3)*4
base_names = ['A'] + list(range(2,11))+['J','K','Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
            cards.extend(str(num) + suit for num in base_names)

In [154]:
deck = pd.Series(card_val, index = cards)

In [155]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [156]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

KS    10
4C     4
8D     8
JH    10
JD    10
dtype: int64

In [157]:
get_suit = lambda card: card[-1] #마지막 글자가 세트 

In [160]:
deck.groupby(get_suit).apply(draw, n=2) # 두장의 카드를 무작위로 뽑자

C  8C     8
   JC    10
D  7D     7
   6D     6
H  6H     6
   KH    10
S  8S     8
   2S     2
dtype: int64

In [161]:
deck.groupby(get_suit, group_keys=False).apply(draw,n=2)

5C      5
KC     10
9D      9
JD     10
JH     10
QH     10
2S      2
10S    10
dtype: int64

In [162]:
df = pd.DataFrame({'category':['a','a','a','a',
                               'b','b','b','b'],
                   'data':np.random.randn(8),
                   'weights':np.random.randn(8)})

In [163]:
df

,category,data,weights
0,a,1.169755,-0.048690
1,a,-0.787150,0.737961
2,a,0.831931,-1.298772
3,a,1.387956,1.591173
4,b,-0.615855,0.485002
5,b,-0.553452,-1.303678
6,b,-0.874806,-0.218766
7,b,0.587036,0.265037


In [164]:
grouped = df.groupby('category')

In [165]:
get_wavg = lambda g : np.average(g['data'],weights=g['weights'])

In [167]:
grouped.apply(get_wavg)

category
a    0.499300
b   -0.996624
dtype: float64

In [168]:
close_px = pd.read_csv('../data/examples/stock_px_2.csv',parse_dates=True, index_col=0)

In [169]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [170]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [174]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [175]:
rets = close_px.pct_change().dropna()

In [176]:
get_year = lambda x : x.year

In [179]:
get_year

<function __main__.<lambda>(x)>

In [177]:
by_year = rets.groupby(get_year)

In [178]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [180]:
import statsmodels.api as sm

In [186]:
def regress(data,yvar,xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [187]:
by_year.apply(regress,'AAPL',['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


In [189]:
tips.pivot_table(index=['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [190]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [191]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],
                 columns='smoker',margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [192]:
tips.pivot_table('tip_pct',index=['time','smoker'],columns='day',
                 aggfunc=len,margins = True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [196]:
tips.pivot_table('tip_pct',index=['time','size','smoker'],
                 columns='day', aggfunc = 'mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706